# Boilerplate

In [1]:
%load_ext autoreload
%autoreload

import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import json
import optuna

from IPython.display import display
from IPython.display import Markdown as md
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

/Users/vpb/anaconda3/envs/kenv/lib/python3.7/site-packages/IPython/extensions/autoreload.py:121: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/vpb/anaconda3/envs/kenv/lib/python3.7/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/Users/vpb/anaconda3/envs/kenv/lib/python3.7/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/Users/vpb/anaconda3/envs/kenv/lib/python3.7/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/Users/vpb/anaconda3/envs/kenv/lib/python3.7/sit

In [2]:
import lightgbm as lgb

from libs.pipeline import Pipe
from libs.tuning import Tuning

In [3]:
df = pd.read_csv("data/UCI_Credit_Card.csv")

df.columns = df.columns.str.lower()

df = df.drop('id', axis = 1)

df = df.rename(columns = {'default.payment.next.month':'target'})

df.to_parquet('data/df_to_feature_selection.parquet', index = False)

In [4]:
df.dtypes.value_counts()

float64    13
int64      11
dtype: int64

# Optuna

In [5]:
tuning = Tuning(df,
                target_var = 'target', 
                n_trials = 30,
                chosen_optimization='f1_score')

[I 2022-10-28 07:11:01,545] A new study created in memory with name: no-name-8b007568-d0ee-4f23-8588-f4491fbf3ef5
[LightGBM] [Fatal] categorical_feature is not a number,
if you want to use a column name,
please add the prefix "name:" to the column name
[W 2022-10-28 07:11:01,577] Trial 0 failed because of the following error: LightGBMError('categorical_feature is not a number,\nif you want to use a column name,\nplease add the prefix "name:" to the column name')
Traceback (most recent call last):
  File "/Users/vpb/anaconda3/envs/kenv/lib/python3.7/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/Users/vpb/Documents/01_ds/git/01_code_snippets/03_credit_risk_modelling_optuna/libs/tuning.py", line 34, in <lambda>
    self.func = lambda trial: self.objective(trial)
  File "/Users/vpb/Documents/01_ds/git/01_code_snippets/03_credit_risk_modelling_optuna/libs/tuning.py", line 91, in objective
    verbose = False
  File "/Users/vpb/a

[LightGBM] [Warning] Unknown parameter: ming_gain_to_split
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l1 is set=85, reg_alpha=0.0 will be ignored. Current value: lambda_l1=85
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.2 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] lambda_l2 is set=100, reg_lambda=0.0 will be ignored. Current value: lambda_l2=100
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


LightGBMError: categorical_feature is not a number,
if you want to use a column name,
please add the prefix "name:" to the column name

In [ ]:
bestparams = tuning.study.best_params

In [ ]:
# O modelo em sí com os parametroz tunados
best = lgb.LGBMClassifier().set_params(**bestparams)

In [ ]:
mbest = Pipe(df, target_var = 'target', model = best)

mbest.popin.scores

mbest.oos.scores